In [1]:
using FFTW
using WAV
using GLMakie

In [2]:
y, Fs, nbits, opts  = wavread("/Users/darwin/Downloads/Unknown-1.wav")

([0.00640888698995941; 0.0043641468550675985; … ; -0.031159398174993132; -0.030671102023377177;;], 44100.0f0, 0x0010, WAVChunk[WAVChunk(Symbol("fmt "), UInt8[0x10, 0x00, 0x00, 0x00, 0x01, 0x00, 0x01, 0x00, 0x44, 0xac, 0x00, 0x00, 0x88, 0x58, 0x01, 0x00, 0x02, 0x00, 0x10, 0x00])])

In [3]:
function difference(x::AbstractArray, τ)
    d = zero(eltype(x))
    N = length(x)
    for j in 1:N-τ
        d += (x[j] - x[j+τ])^2
    end
    return d
end

difference (generic function with 1 method)

In [4]:
function first_local_minimum(x)
    for i in 2:length(x)-1
        if x[i-1] > x[i] && x[i] < x[i+1]
            return (i, x[i])
        end
    end
    return nothing
end

first_local_minimum (generic function with 1 method)

In [5]:
function estimate_pitch(y, Fs, f_min = 30, f_max = 150)
    volume = sum(abs.(y))/length(y)
    if volume < 0.01
        return nothing
    end
    range = floor(Int, Fs/f_max):ceil(Int, Fs/f_min)
    d = [difference(y, j) for j ∈ range]
    d′ = [j*d[j]/sum(d[1:j]) for j in 1:length(range)]

    ind = findfirst(x -> x < 0.5, d′)

    τ = ind + first_local_minimum(d′[ind:end])[1] - 1
    interpolation = (d′[τ - 1] - d′[τ + 1])/(2*(2*d′[τ] - d′[τ - 1] - d′[τ + 1]))

    τ += range[1] - 1


    return Fs/(τ + interpolation)

end

estimate_pitch (generic function with 3 methods)

In [6]:
f = rand()*(150-30) + 30

x = 0:1/Fs:1
ys = sin.(2π*f.*x)
for _ in 1:rand(2:5)
    c = rand()
    ys .+= sin.(2π*f.*x)
end

In [7]:
f_estimate = estimate_pitch(ys, Fs)

45.744054791431964

In [8]:
1200*log2(f/f_estimate)

0.2131779183847723